In [ ]:
# Installing Packages

import os
import subprocess
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, levene, ranksums
from sklearn.linear_model import LinearRegression
import numpy as np
import pyBigWig
import math
import re
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.multitest import multipletests
from statsmodels.multivariate.manova import MANOVA
from scipy import stats
import statsmodels.api as sm
from matplotlib import gridspec
from matplotlib.patches import Patch
import matplotlib.colors as mcolors
import glob
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [13]:
df = pd.read_csv(
    "/labmed/workspace/lotta/finaletoolkit/output_workflow/EE86268_frag_intervals.bed",
    sep="\t",
    header=None,
    names=["chrom", "start", "stop", "name", "mean", "median", "stdev", "min", "max"]
)
df = df.iloc[1:].reset_index(drop=True)

print(df.head())

  chrom    start     stop name                mean median               stdev  \
0  chr1   920000   925000    .  118.50909090909092  126.0   39.44334151770923   
1  chr1  1070000  1075000    .  117.85416666666667  129.0  44.682672621933534   
2  chr1  1165000  1170000    .  126.26829268292683  126.0  37.217897864808435   
3  chr1  1170000  1175000    .  132.52857142857144  141.0   61.34812173829226   
4  chr1  1175000  1180000    .   128.2340425531915  138.0   51.60084642467476   

  min  max  
0  40  179  
1  43  175  
2  39  176  
3  36  332  
4  41  304  


In [14]:
# 1. Bin-Wide-Analysis, Binning the genome, bin size 5Mb

bin_size = 5000000
df["start"] = df["start"].astype(int)
df["stop"] = df["stop"].astype(int)

num_cols = ["mean", "median", "stdev", "min", "max"]
df[num_cols] = df[num_cols].astype(float)
df['bin'] = df['start'] // bin_size
df['sample'] = "EE86268"
df['group'] = "cancer"
print(df.head())



  chrom    start     stop name        mean  median      stdev   min    max  \
0  chr1   920000   925000    .  118.509091   126.0  39.443342  40.0  179.0   
1  chr1  1070000  1075000    .  117.854167   129.0  44.682673  43.0  175.0   
2  chr1  1165000  1170000    .  126.268293   126.0  37.217898  39.0  176.0   
3  chr1  1170000  1175000    .  132.528571   141.0  61.348122  36.0  332.0   
4  chr1  1175000  1180000    .  128.234043   138.0  51.600846  41.0  304.0   

   bin   sample   group  
0    0  EE86268  cancer  
1    0  EE86268  cancer  
2    0  EE86268  cancer  
3    0  EE86268  cancer  
4    0  EE86268  cancer  


In [15]:
binned_df = (
    df.groupby(['sample', 'group', 'chrom', 'bin'])
      .agg({
          "mean": "mean",
          "median": "mean",
          "stdev": "mean",
          "min": "mean",
          "max": "mean"
      })
      .reset_index()
)

print(binned_df.head())

    sample   group chrom  bin        mean      median      stdev        min  \
0  EE86268  cancer  chr1    0  130.269036  127.757576  54.243433  43.393939   
1  EE86268  cancer  chr1    1  130.746768  136.062500  52.689322  41.062500   
2  EE86268  cancer  chr1    2  129.756315  131.230769  47.088429  50.076923   
3  EE86268  cancer  chr1    3  123.922138  133.035714  48.017148  42.071429   
4  EE86268  cancer  chr1    4  124.930246  130.642857  51.114649  39.500000   

          max  
0  277.515152  
1  283.312500  
2  267.846154  
3  254.785714  
4  284.214286  
